In [1]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

## 피봇테이블

- 가지고 있는 데이터원본을 원하는 형태의 가공된 정보를 보여주는 것
    - 자료의 형태를 변경하기 위해 많이 사용하는 방법  
    

- 좌측표는 제품이 생산될 때 마다 코드, 크기, 생산 수량을 기록 오른쪽은 지역별로 제품생산코드를 요약하여 어떤 제품을 몇번 생산했는지 요약
![](../picture/피벗1.png)


- 좌측표는 제품이 생산될 때 마다 코드, 크기, 생산 수량을 기록 오른쪽은 제품 크기별로 각 제품이 몇번 생산 되었는지 요약
![](../picture/피벗2.png)


- 방법 : 두개의 키를 사용해서 데이터를 선택

- pivot_table(data,values=None,index=None,columns=None,aggfunc='mean',margins=False,margins_name='All')

    - data : 분석할 데이터 프레임. 메서드 형식일때는 필요하지 않음 ex)df1.pivot_table()
    - values : 분석할 데이터 프레임에서 분석할 열
    - index :  행 인덱스로 들어갈 키열 또는 키열의 리스트
    - columns : 열 인덱스로 들어갈 키열 또는 키열의 리스트
    - fill_value : NaN이 표출될 때 대체값 지정
    - margins : 모든 데이터를 분석한 결과를 행으로 표출할 지 여부
    - margins_name : margins가 표출될 때 그 열(행)의 이름`
    - aggfunc : 집계함수 (요약 결과에 적용시킬 함수)


## 피봇테이블을 작성할 때 반드시 설정해야 되는 인수
- data : 사용 데이터 프레임
- index : 행 인덱스로 사용할 필드(기준 필드로 작용됨)
- 인덱스 명을 제외한 나머지 값(data)은 수치 data 만 사용함
- 기본 함수가 평균(mean)함수 이기 때문에 각 데이터의 평균값이 반환


In [2]:
data = {
    "도시": ["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}

columns = ["도시", "연도", "인구", "지역"]
df1 = pd.DataFrame(data, columns=columns)
df1


,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [6]:
# 각 도시에 대한 연도별 인구 수 평균

df1.pivot('도시','연도','인구')
# 평균을 구할 수 없어서 평균없이 출력

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


In [7]:
# 각 지역별 연도별 인구수 평균

df1.pivot(['지역', '도시'], '연도', '인구')
# index -> [지역 + 도신], column -> 연도, value=인구 -> 값, aggfunc=mean (생략)

연도           2005       2010       2015
지역  도시                                 
경상권 부산  3512547.0  3393191.0  3448737.0
수도권 서울  9762546.0  9631482.0  9904312.0
    인천        NaN   263203.0  2890451.0

## pivot_table 예제 : titanic 데이터 사용
- pivot은 연산 불가능 하지만 pivot_table은 연산 가능

In [8]:
import seaborn as sns

df = sns.load_dataset('titanic')[['age','sex','class','fare','survived']]

df.shape
df.head()


(891, 5)

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0


In [9]:
# 각 선실 등급별로 숙박객의 평균 나이


pdf1 = pd.pivot_table(df,           #피벗할 데이터 플레임
                     index='class', #행 위치에 들어갈 열
                     values='age')  #계산 열
pdf1

,age
class,
First,38.233441
Second,29.877630
Third,25.140620


In [12]:
# 각 선실 등급별로 숙박객의  성별 평균 나이


pdf1 = pd.pivot_table(df,index='class', columns='sex', values='age') 
pdf1

sex,female,male
class,,
First,34.611765,41.281386
Second,28.722973,30.740707
Third,21.750000,26.507589


In [13]:
# 각 선실 등급별 숙박객의 생존자 수와 생존률을 성별로 요약

pdf2=pd.pivot_table(df, index='class', columns='sex', values='survived', aggfunc=['mean','sum'])
pdf2

mean              sum     
sex       female      male female male
class                                 
First   0.968085  0.368852     91   45
Second  0.921053  0.157407     70   17
Third   0.500000  0.135447     72   47

In [16]:
# 각 선실 등급에 따른 남/여 생존 여부별 나이와 티케값 평균과 최대값을 요약

pdf3 = pd.pivot_table(df, index=['class','sex'], 
                      columns='survived', 
                      values=['age','fare'], 
                      aggfunc=['mean', 'max'])
pdf3

mean                                      max        \
                     age                   fare               age         
survived               0          1           0           1     0     1   
class  sex                                                                
First  female  25.666667  34.939024  110.604167  105.978159  50.0  63.0   
       male    44.581967  36.248000   62.894910   74.637320  71.0  80.0   
Second female  36.000000  28.080882   18.250000   22.288989  57.0  55.0   
       male    33.369048  16.022000   19.488965   21.095100  70.0  62.0   
Third  female  23.818182  19.329787   19.773093   12.464526  48.0  63.0   
       male    27.255814  22.274211   12.204469   15.579696  74.0  45.0   

                                 
                 fare            
survived            0         1  
class  sex                       
First  female  151.55  512.3292  
       male    263.00  512.3292  
Second female   26.00   65.0000  
       male     73.50   39.0000  
Third  female   69.55   31.3875  
       male     69.55   56.4958

## 그룹 분석
- 만약 키가 지정하는 조건에 맞는 데이터가 하나 이상이라서 데이터 그룹을 이루는 경우에는 그룹의 특성을 보여주는 그룹분석(group analysis)을 해야 함

    - 그룹분석은 피봇테이블과 달리 키에 의해서 결정되는 데이터가 여러개가 있을 경우 미리 지정한 연산을 통해 그 그룹 데이터의 대표값을 계산 하는 것


- 판다스에서는 groupby 메서드를 사용하여 아래 내용 처럼 그룹분석을 진행

    - 분석하고자 하는 시리즈나 데이터프레임에 groupby 메서드를 호출하여 그룹화 수행

    - 그룹 객체에 대해 그룹연산을 수행


## groupby 메서드¶
- groupby 메서드는 데이터를 그룹 별로 분류하는 역할을 함 

- groupby 메서드의 인수

    - 열 또는 열의 리스트

    - 행 인덱스

- 연산 결과로 그룹 데이터를 나타내는 GroupBy 클래스 객체를 반환
    - 이 객체에는 그룹별로 연산을 할 수 있는 그룹연산 메서드가 있음


## GroupBy 클래스 객체의 그룹연산 메서드

- size, count: 그룹 데이터의 갯수

- mean, median, min, max: 그룹 데이터의 평균, 중앙값, 최소, 최대

- sum, prod, std, var, quantile : 그룹 데이터의 합계, 곱, 표준편차, 분산, 사분위수

- first, last: 그룹 데이터 중 가장 첫번째 데이터와 가장 나중 데이터


In [17]:
np.random.seed(0)
df2 = pd.DataFrame({
    'key1': ['A', 'A', 'B', 'B', 'A'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': [1, 2, 3, 4, 5],
    'data2': [10, 20, 30, 40, 50]
})
df2


,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


In [18]:
groups = df2.groupby(df2.key1)
groups

In [19]:
groups.groups #groups 속성으로 그룹 인덱스 확인

{'A': [0, 1, 4], 'B': [2, 3]}

In [42]:
pd.DataFrame(groups)

,0,1
0,A,key1 key2 data1 data2 0 A one 1 ...
1,B,key1 key2 data1 data2 2 B one 3 ...


In [26]:
pd.DataFrame(groups).loc[0].values

array(['A',   key1 key2  data1  data2
            0    A  one      1     10
            1    A  two      2     20
            4    A  one      5     50], dtype=object)

In [27]:
pd.DataFrame(groups).loc[1].values

array(['B',   key1 key2  data1  data2
            2    B  one      3     30
            3    B  two      4     40], dtype=object)

In [28]:
pd.DataFrame(groups).loc[0]

0                                                    A
1      key1 key2  data1  data2
0    A  one      1  ...
Name: 0, dtype: object

In [23]:
groups.sum()

,data1,data2
key1,,
A,8,80
B,7,70


In [29]:
groups['data1'].sum() # 특정 컬럼 추출 후에 sum 연산 진행

groups[['data1']].sum()

key1
A    8
B    7
Name: data1, dtype: int64

,data1
key1,
A,8
B,7


In [25]:
groups.sum()['data1'] # 모든 컬럼에 대해 sum 연산 진행 후에 특정 컬럼 추출
groups.sum()[['data1']]

key1
A    8
B    7
Name: data1, dtype: int64

,data1
key1,
A,8
B,7


## 이 외에도 많이 사용되는 그룹 연산

- agg, aggregate

    - 만약 원하는 그룹연산이 없는 경우 함수를 만들고 이 함수를 agg에 전달한다.

    - 또는 여러가지 그룹연산을 동시에 하고 싶은 경우 함수 이름 문자열의 리스트를 전달한다.

- describe

    - 하나의 그룹 대표값이 아니라 여러개의 값을 데이터프레임으로 구한다.

- apply

    - describe 처럼 하나의 대표값이 아닌 데이터프레임을 출력하지만 원하는 그룹연산이 없는 경우에 사용한다.

- transform

    - 그룹에 대한 대표값을 만드는 것이 아니라 그룹별 계산을 통해 데이터 자체를 변형한다.


## 그룹 함수 예제

- apply()/agg() (동일한 기능)


    - DF 등에 벡터라이징 연산을 적용하는 함수(axis = 0/1 이용하여 행/열 적용가능)
    - agg 함수는 숫자 타입의 스칼라만 리턴하는 함수를 적용하는 apply의 특수한 경우
        -  스칼라 : 하나의 수치(數値)만으로 완전히 표시되는 양. 방향의 구별이 없는 물리적 수량임. 질량·에너지·밀도(密度)·전기량(電氣量) 따위.
        
- 차이 하나의 값만 return할 경우 agg 나머지 특수한 경우에 apply 사용


In [32]:
import seaborn as sns
iris = sns.load_dataset("iris")


In [33]:
iris.shape
iris.head()
iris

(150, 5)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [34]:
iris['species'].value_counts()

virginica     50
setosa        50
versicolor    50
Name: species, dtype: int64

In [38]:
i_groups=iris.groupby(iris.species) #iris 품종별로 그룹화

In [36]:
i_groups.sum()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,250.3,171.4,73.1,12.3
versicolor,296.8,138.5,213.0,66.3
virginica,329.4,148.7,277.6,101.3


In [ ]:
## 사용자 정의 함수

## agg 함수

In [40]:
def peak_to_peak_ration(x):
    return x.max()/x.min() # 함수 반환 갑이 수치 스칼라 타입1


In [41]:
i_groups.agg(peak_to_peak_ration) #품종에 따른 특성(컬럼) 별로 사용자 정의함수 연산을 적용

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


## apply함수 

In [42]:
def top3_petal_length(df):
    return df.sort_values(by="petal_length", ascending=False)[:3] # 함수 반환값이 수치 집합
# 꽃이 길이가 가장 긴 3개의 행을 넘겨 줌


In [43]:
iris.groupby(iris.species).apply(top3_petal_length)

sepal_length  sepal_width  petal_length  petal_width  \
species                                                                
setosa     24            4.8          3.4           1.9          0.2   
           44            5.1          3.8           1.9          0.4   
           23            5.1          3.3           1.7          0.5   
versicolor 83            6.0          2.7           5.1          1.6   
           77            6.7          3.0           5.0          1.7   
           72            6.3          2.5           4.9          1.5   
virginica  118           7.7          2.6           6.9          2.3   
           117           7.7          3.8           6.7          2.2   
           122           7.7          2.8           6.7          2.0   

                   species  
species                     
setosa     24       setosa  
           44       setosa  
           23       setosa  
versicolor 83   versicolor  
           77   versicolor  
           72   versicolor  
virginica  118   virginica  
           117   virginica  
           122   virginica

## agg apply 비교
- agg는 집합적 자료에 사용X

In [44]:
i_groups.agg(peak_to_peak_ration) #품종에 따른 특성(컬럼) 별로 사용자 정의함수 연산을 적용
i_groups.apply(peak_to_peak_ration)


,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


In [47]:
iris.groupby(iris.species).apply(top3_petal_length)
iris.groupby(iris.species).agg(top3_petal_length)## 에러 발생  agg는 집합적 데이터에 적용 X

sepal_length  sepal_width  petal_length  petal_width  \
species                                                                
setosa     24            4.8          3.4           1.9          0.2   
           44            5.1          3.8           1.9          0.4   
           23            5.1          3.3           1.7          0.5   
versicolor 83            6.0          2.7           5.1          1.6   
           77            6.7          3.0           5.0          1.7   
           72            6.3          2.5           4.9          1.5   
virginica  118           7.7          2.6           6.9          2.3   
           117           7.7          3.8           6.7          2.2   
           122           7.7          2.8           6.7          2.0   

                   species  
species                     
setosa     24       setosa  
           44       setosa  
           23       setosa  
versicolor 83   versicolor  
           77   versicolor  
           72   versicolor  
virginica  118   virginica  
           117   virginica  
           122   virginica

ValueError: Shape of passed values is (5, 3), indices imply (4, 3)

## apply 예제

In [49]:
def q3cut(s):
    return pd.qcut(s, 3, labels=['소','중','대']).astype(str)

In [50]:
iris.groupby(iris.species).petal_length.apply(q3cut)

0      소
1      소
2      소
3      중
4      소
      ..
145    소
146    소
147    소
148    중
149    소
Name: petal_length, Length: 150, dtype: object

In [51]:
iris["petal_length"] = iris.groupby(iris.species).petal_length.apply(q3cut)
## 품종 별로 나누어서 각각 나누는 방식
iris.head(10)
iris.tail(10)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,소,0.2,setosa
1,4.9,3.0,소,0.2,setosa
2,4.7,3.2,소,0.2,setosa
3,4.6,3.1,중,0.2,setosa
4,5.0,3.6,소,0.2,setosa
5,5.4,3.9,대,0.4,setosa
6,4.6,3.4,소,0.3,setosa
7,5.0,3.4,중,0.2,setosa
8,4.4,2.9,소,0.2,setosa
9,4.9,3.1,중,0.1,setosa


,sepal_length,sepal_width,petal_length,petal_width,species
140,6.7,3.1,중,2.4,virginica
141,6.9,3.1,소,2.3,virginica
142,5.8,2.7,소,1.9,virginica
143,6.8,3.2,대,2.3,virginica
144,6.7,3.3,중,2.5,virginica
145,6.7,3.0,소,2.3,virginica
146,6.3,2.5,소,1.9,virginica
147,6.5,3.0,소,2.0,virginica
148,6.2,3.4,중,2.3,virginica
149,5.9,3.0,소,1.8,virginica


## 그룹함수 및 피봇 테이블 이용 간단한 분석 예제


#### 식당에서 식사 후 내는 팁(tip)과 관련된 데이터이용

- seaborn 패키지 내 tips 데이터셋 사용

    - total_bill: 식사대금

    - tip: 팁

    - sex: 성별

    - smoker: 흡연/금연 여부

    - day: 요일

    - time: 시간

    - size: 인원


In [52]:
tips = sns.load_dataset("tips")

tips.shape
tips.tail()


(244, 7)

,total_bill,tip,sex,smoker,day,time,size
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2
243,18.78,3.00,Female,No,Thur,Dinner,2


### 식사 대금 팁의 비율이 어느 경우에 높은지?
- 가공 컬럼 생성 : 식사대금 대비 팁의 비율
- tip_pt = 팁 / 식사대금


In [53]:
tips['tip_pt'] = tips['tip']/tips['total_bill']
tips

,total_bill,tip,sex,smoker,day,time,size,tip_pt
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204


In [54]:
tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
 7   tip_pt      244 non-null    float64 
dtypes: category(4), float64(3), int64(1)
memory usage: 9.3 KB


In [55]:
tips.describe()

,total_bill,tip,size,tip_pt
count,244.000000,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672,0.160803
std,8.902412,1.383638,0.951100,0.061072
min,3.070000,1.000000,1.000000,0.035638
25%,13.347500,2.000000,2.000000,0.129127
50%,17.795000,2.900000,2.000000,0.154770
75%,24.127500,3.562500,3.000000,0.191475
max,50.810000,10.000000,6.000000,0.710345


In [59]:
# 성별 인원수 계산
tips.groupby('sex').count() #결측치가 없어서 동일한 결과


,total_bill,tip,smoker,day,time,size,tip_pt
sex,,,,,,,
Male,157,157,157,157,157,157,157
Female,87,87,87,87,87,87,87


In [60]:
tips.groupby(['sex','smoker']).size()

sex     smoker
Male    Yes       60
        No        97
Female  Yes       33
        No        54
dtype: int64

In [58]:
# 흡연 유무에 따른 성별 인원을 피봇 테이블로 구현

tips.pivot_table('total_bill', index='sex', columns='smoker', aggfunc='count')

smoker,Yes,No
sex,,
Male,60,97
Female,33,54


In [128]:
# 성별로 팁 비율의 평균

tips.groupby('sex')['tip_pt'].mean() # 여성이 식사금액 대비 팁 비율의 평균이 근속하게 높음

sex
Male      0.157651
Female    0.166491
Name: tip_pt, dtype: float64

In [129]:
# 흡연 유무에 따른 팁 비율의 평균

tips.groupby('smoker')['tip_pt'].mean() #흡연자가 비흡연자에 비해 팀 비율이 근속하게 높음

smoker
Yes    0.163196
No     0.159328
Name: tip_pt, dtype: float64

In [130]:
# 성별과 흡연 유무에 따른 팀 비율의 평균 - 피봇테이블로 확인
tips.pivot_table('tip_pt', index='sex', columns='smoker')

smoker,Yes,No
sex,,
Male,0.152771,0.160669
Female,0.182150,0.156921


## 여성이면서 흡연자의 팁 비율이 높게 나타났음 - 여성 흡연자가 팁을 많이줌


In [61]:
# 평균 통계량만 확인했으므로 다른 통계도 확인
tips.groupby(['sex','smoker'])[['tip_pt']].describe()

tip_pt                                                    \
               count      mean       std       min       25%       50%   
sex    smoker                                                            
Male   Yes      60.0  0.152771  0.090588  0.035638  0.101845  0.141015   
       No       97.0  0.160669  0.041849  0.071804  0.131810  0.157604   
Female Yes      33.0  0.182150  0.071595  0.056433  0.152439  0.173913   
       No       54.0  0.156921  0.036421  0.056797  0.139708  0.149691   

                                   
                    75%       max  
sex    smoker                      
Male   Yes     0.191697  0.710345  
       No      0.186220  0.291990  
Female Yes     0.198216  0.416667  
       No      0.181630  0.252672